# Data Component Use Case for Landslide Susceptibility Calculation

## Introduction

Landslide susceptibility is the likelihood of a landslide occurring in an area on the basis of local terrain condition to estimate “where” landslides are likely to occur. This Jupyter notebook demonstrates how to use several [CSDMS data components](https://csdms.colorado.edu/wiki/DataComponents) to download topography and soil datasets to calculate the landslide susceptibility for a study area in Puerto Rico. 

In this notebook, it includes the following sections:
- [Step0: Initial Setup](#setup)
  
  This section will help install several Python packages and create the input/output folders.
  <br>
- [Step1: Download Datasets](#step1) 

  This section will download the topography and soil datasets.
  <br>
- [Step2: Regrid Datasets](#step2)

  This section will regrid the datasets in the same grid resolution. 
  <br>
- [Step3: Calculate Susceptibility](#step3)

  This section will use the datasets to calculate the landslide susceptibility.
  <br>
- [Step4: Visualize Results](#step4)

  This section will visualize the final results as a short video.
  <br>
  
**Run Notebook**: You can test and run this Jupyter Notebook through [HydroShare](https://www.hydroshare.org/resource/df5fa2f5d1b74be4bf0a049e1e59889c/). If you have any suggestion to improve this notebook, please create a Github issue [here](https://github.com/gantian127/landslide_usecase).

**Suggested Citation**: Gan, T., Campforts, B., Tucker, G. E., Overeem, I. (2022). Data Component Use Case for Landslide Susceptibility Calculation, HydroShare, https://www.hydroshare.org/resource/df5fa2f5d1b74be4bf0a049e1e59889c/

<a id='setup'></a>
## Initial Setup

We will install the following packages for this notebook. There are two installation options that you can choose based on whether you will run the notebook on your local PC or in HydroShare. After you install the packages, you will need to run a helper function to create API Key files for the ERA5 and Topography data components.
- pymt
- cdsapi
- pymt-era5
- pymt-topography
- numpy
- xesmf
- xarray
- rioxarray
- cftime
- imageio
- imageio_ffmpeg
- matplotlib

### Install Packages (on CUAHSI JupyterHub)

If you access this notebook from [HydroShare](https://www.hydroshare.org/resource/df5fa2f5d1b74be4bf0a049e1e59889c/) and run it on the CUAHSI JupyterHub, please run the command below to install the required packages.

In [ ]:
! conda install -y --file=requirements.txt
! pip install pymt-era5 imageio_ffmpeg

### Install Packages (on Local PC)
If you want to run this notebook on your PC, you can run the following command which will create a separate conda environment named "landslide_usecase" and install all the required packages for you. After the installation, please make sure to activate the environment to run this notebook.

In [ ]:
! conda env create --file=environment.yml

### Install API key files
For the ERA5 and Topography data components, there is a need to create API key files to download the datasets. The install_api_key( ) function will ask for your [CDS API Key](https://cds.climate.copernicus.eu/api-how-to) and [Open Topography API Key](https://opentopography.org/blog/introducing-api-keys-access-opentopography-global-datasets) to create API key files. Please make sure you have already obtained those API Keys before you run this helper function. 


In [ ]:
from utils import install_api_key
install_api_key()

### Create folders
We will create three folders for this notebook:
- **configuration file folder**: this folder includes several configuration files which will be used by the data components. In this example, we have prepared these configuration files and put them in this folder. 
- **cache folder**: this folder stores the downloaded data files.
- **results folder**: this folder stores the final results.

In [ ]:
import os

study_area = 'puerto_rico'

config_dir = os.path.join(os.getcwd(), 'config_files_{}'.format(study_area)) # put config file in it
results_dir = os.path.join(os.getcwd(), 'results_{}'.format(study_area)) # create a folder
cache_dir = os.path.join(os.getcwd(),'cache_{}'.format(study_area))


for folder in [config_dir, results_dir, cache_dir]:
    if not os.path.isdir(folder):
        os.mkdir(folder)
        print(folder)

<a id='step1'></a>
## Step 1 Download Datasets

### Background 

Landslides are a frequent hazard in Puerto Rico which are mainly caused by the steep terrain and heavy rainfall from hurricanes and other tropical weather systems. For example, Hurricane Maria hit the island of Puerto Rico on September 20th, 2017 and triggered more than 40,000 landslides in Puerto Rico (see details [here](https://www.usgs.gov/supplemental-appropriations-for-disaster-recovery-activities/landslides-triggered-hurricane-maria)). The map below shows concentration and distribution of landslides generated by rainfall associated with Hurricane Maria in Puerto Rico (map source from [USGS](https://www.usgs.gov/supplemental-appropriations-for-disaster-recovery-activities/assessment-landslide-and-debris-flow)).

In this example, we will calculate the hourly landslide susceptibility for the area that has high concentration of landslide of Puerto Rico during Hurricane Maria. We will prepare the following datasets to calculate the landslide susceptibility. Details for how to calculate the landslide susceptibility is described in [Step 3](#step3)
- OpenTopography DEM 
- ERA5 volumetric soil water and precipitation 
- Soil depth  
- Slope angle

<img src="https://github.com/gantian127/landslide_usecase/blob/master/Landslide_map.jpeg?raw=true" width="900">

### OpenTopography DEM 

We will use the [Topography data component](https://csdms.colorado.edu/wiki/Model:Topography_Data_Component) to download the Digital Elevation Model (DEM) data with 90m resolution. The 'dem_config.yaml' file includes the parameter settings of this data component. The following cells demonstrate how to use the configuration file to initialize a data component and how to use the variable and grid related methods of this data component to get the metadata as well as the data values.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from pymt.models import Topography

In [ ]:
# initialize Topography data component
dem = Topography()
dem.initialize(os.path.join(config_dir, 'dem_config.yaml'))

In [ ]:
# get DEM variable info
var_name = dem.output_var_names[0]
var_unit = dem.var_units(var_name)
var_location = dem.var_location(var_name)
var_type = dem.var_type(var_name)
var_grid = dem.var_grid(var_name)
var_itemsize = dem.var_itemsize(var_name)
var_nbytes = dem.var_nbytes(var_name)
print('variable_name: {} \nvar_unit: {} \nvar_location: {} \nvar_type: {} \nvar_grid: {} \nvar_itemsize: {}' 
            '\nvar_nbytes: {} \n'. format(var_name, var_unit, var_location, var_type, var_grid, var_itemsize, var_nbytes))

In [ ]:
# get DEM grid info 
dem_grid_ndim = dem.grid_ndim(var_grid) 
dem_grid_type = dem.grid_type(var_grid)
dem_grid_shape = dem.grid_shape(var_grid)
dem_grid_spacing = dem.grid_spacing(var_grid)
dem_grid_origin = dem.grid_origin(var_grid)

print('grid_ndim: {} \ngrid_type: {} \ngrid_shape: {} \ngrid_spacing: {} \ngrid_origin: {}'.format(
    dem_grid_ndim, dem_grid_type, dem_grid_shape, dem_grid_spacing, dem_grid_origin))

In [ ]:
# get DEM variable data
dem_data = dem.get_value(var_name)
dem_data_2D = dem_data.reshape(dem_grid_shape)

# get X, Y extent for plot
min_y, min_x = dem_grid_origin
max_y = min_y + dem_grid_spacing[0]*(dem_grid_shape[0]-1)
max_x = min_x + dem_grid_spacing[1]*(dem_grid_shape[1]-1)
dy = dem_grid_spacing[0]/2
dx = dem_grid_spacing[1]/2
dem_extent = [min_x - dx, max_x + dx, min_y - dy, max_y + dy]

# plot DEM data
fig, ax = plt.subplots(1,1, figsize=(16,4))
im = ax.imshow(dem_data_2D, extent=dem_extent)
ax.title.set_text('Topography Data (m)')
ax.set_xlabel('longitude [degree_east]')
ax.set_ylabel('latitude [degree_north]')
fig.colorbar(im)
ax.title

### ERA5 Volumetric Soil Water & Precipitation 
We will use the [ERA5 data component](https://csdms.colorado.edu/wiki/Model:ERA5_Data_Component) to download the hourly volumetric soil water data and the precipitation data of the study area with 0.25 degrees (27-28km) resolution. The volumetric soil water data will be used for calculating the susceptibility, while the precipitation data is mainly used for results visualization. 

The 'era5_config.yaml' file includes the parameter settings of this data component. The following cells demonstrate how to use the configuration file to initialize an ERA5 data component and how to use the variable, grid and time related methods to get the metadata as well as the data values. You'll notice that although the ERA5 and Topography data components download the datasets from different sources, they are using the same methods to get information from the datasets. Please note that sometimes the request for ERA5 data may be queued which may take longer time (>10min) to get the data downloaded.

In [ ]:
from pymt.models import Era5

# initialize ERA5 data component
era5 = Era5()
era5.initialize(os.path.join(config_dir,'era5_config.yaml'))

In [ ]:
# get ERA5 variable info
for var_name in era5.output_var_names:
    var_unit = era5.var_units(var_name)
    var_location = era5.var_location(var_name)
    var_type = era5.var_type(var_name)
    var_grid = era5.var_grid(var_name)
    var_itemsize = era5.var_itemsize(var_name)
    var_nbytes = era5.var_nbytes(var_name)
    print('variable_name: {} \nvar_unit: {} \nvar_location: {} \nvar_type: {} \nvar_grid: {} \nvar_itemsize: {}' 
            '\nvar_nbytes: {} \n'. format(var_name, var_unit, var_location, var_type, var_grid, var_itemsize, var_nbytes))

In [ ]:
# get ERA5 grid info
era5_grid_ndim = era5.grid_ndim(var_grid) 
era5_grid_type = era5.grid_type(var_grid)
era5_grid_shape = era5.grid_shape(var_grid)
era5_grid_spacing = era5.grid_spacing(var_grid)
era5_grid_origin = era5.grid_origin(var_grid)

print('grid_ndim: {} \ngrid_type: {} \ngrid_shape: {} \ngrid_spacing: {} \ngrid_origin: {}'.format(
    era5_grid_ndim, era5_grid_type, era5_grid_shape, era5_grid_spacing, era5_grid_origin))

In [ ]:
# get ERA5 time info
era5_start_time = era5.start_time
era5_end_time = era5.end_time
era5_time_step = era5.time_step
era5_time_unit = era5.time_units
era5_time_steps = int((era5_end_time - era5_start_time)/era5_time_step) + 1

print('start_time:{} \nend_time:{} \ntime_step:{} \ntime_unit:{} \ntime_steps:{}'.format(
    era5_start_time, era5_end_time, era5_time_step, era5_time_unit, era5_time_steps))

In [ ]:
# get ERA5 variables data and plot (for the first time step)
fig = plt.figure(figsize=(16,12)) 
nrows, ncols = 3, 2
i = 1

for var_name in era5.output_var_names:
    ax = fig.add_subplot(nrows, ncols, i)
    var_unit = era5.var_units(var_name)
    
    # get variable data    
    era5_data = era5.get_value(var_name)
    era5_data_2D = era5_data.reshape(era5_grid_shape)
    
    # get X, Y extent for plot
    min_y, min_x = era5_grid_origin
    max_y = min_y + era5_grid_spacing[0]*(era5_grid_shape[0]-1)
    max_x = min_x + era5_grid_spacing[1]*(era5_grid_shape[1]-1)
    dy = era5_grid_spacing[0]/2
    dx = era5_grid_spacing[1]/2
    era5_extent = [min_x - dx, max_x + dx, min_y - dy, max_y + dy]

    # plot data
    im = ax.imshow(era5_data_2D, extent=era5_extent, cmap='Blues')
    ax.title.set_text('{} ({})'.format(var_name,var_unit ))
    ax.set_xlabel('longitude [degree_east]')
    ax.set_ylabel('latitude [degree_north]')
    cbar = plt.colorbar(im, ax=ax)
    
    i += 1

### Soil Depth data

Since the data component for soil depth data is not available yet, we will use the xarray and rioxarray to subset and download the soil depth data with 250m resolution from [SoilGrids](https://www.isric.org/explore/soilgrids) system. In this dataset, the maximum soil depth value is 200cm. Grid with values larger than 200cm represents open water area. 

In [ ]:
import rioxarray 
import xarray

# download soil depth data
soil_raster = xarray.open_rasterio("https://files.isric.org/soilgrids/former/2017-03-10/data/BDRICM_M_250m_ll.tif")
soil_depth_data = soil_raster.rio.clip_box(
    minx=-67.13,
    miny=18.02,
    maxx=-66.13,
    maxy=18.32,
)

In [ ]:
# plot soil depth data
soil_depth_data.plot(figsize=(16,4))
soil_depth_data.rio.to_raster(os.path.join(cache_dir,'soil_depth.tif'))
plt.title('Soil Depth (cm)')

### Slope angle

Slope angle is one of the input for calculating the landslide susceptibility. In this example, we will use the Topography data and the [RasterModelGrid](https://landlab.readthedocs.io/en/master/reference/grid/raster.html) from [Landlab](https://landlab.readthedocs.io/en/master/index.html) to calculate the slope angle for the study area. 

In [ ]:
from landlab import RasterModelGrid

# calculate slope angle using Topography data
model_grid = RasterModelGrid(dem_data_2D.shape,xy_spacing=(90,90))
slope = model_grid.calc_slope_at_node(elevs=dem_data) # slope in magintude, 1D array
slope_angle = slope.reshape(dem_data_2D.shape) # reshape as 2D array

In [ ]:
# plot slope angle
fig, ax = plt.subplots(figsize=(16,4))
im=ax.imshow(slope_angle, extent=dem_extent)
cbar = fig.colorbar(im)
ax.title.set_text('Slope Angle (radiance)')
ax.set_xlabel('longitude [degree_east]')
ax.set_ylabel('latitude [degree_north]')

<a id='step2'></a>
## Step 2 Regrid datasets

Before calculating the landslide susceptibility, we need to regrid all the datasets in the same grid resolution. We will take the Topography dataset as the template to regrid the soil depth and the ERA5 datasets. This will make these datasets to have the same resolution as the Topography dataset. Since all the original datasets are in the WGS84 geographic coordinate system, there is no need for data reprojection. Because ERA5 datasets are multidimensional space time data, we will regrid ERA5 datasets in a for loop in [Step3](#step3). In this example, we will use the regrid_data( ) function for regridding purpose. 



In [ ]:
import xarray
import xesmf as xe

from utils import regrid_data

# define destination grid coordinate using Topography data
dem_y = np.flip(np.arange(dem_grid_shape[0])*dem_grid_spacing[0] + dem_grid_origin[0])
dem_x = np.arange(dem_grid_shape[1])*dem_grid_spacing[1] + dem_grid_origin[1]
dest_coor = {'lon': dem_x, 
             'lat': dem_y}

In [ ]:
# regrid soil depth data
soil_depth_coor = {'lon': soil_depth_data.x.values,
                   'lat': soil_depth_data.y.values}

soil_depth = regrid_data(soil_depth_data.values[0], soil_depth_coor, dest_coor) 
soil_depth = soil_depth/100 # units conversion as m

In [ ]:
# plot result
fig, ax = plt.subplots(1,1,figsize=(16,4))
im = ax.imshow(soil_depth, extent=dem_extent)
ax.title.set_text('Regridded Soil Depth Data (m)')
ax.set_xlabel('longitude [degree_east]')
ax.set_ylabel('latitude [degree_north]')
cbar = plt.colorbar(im)

<a id='step3'></a>
## Step 3  Calculate Susceptibility

This section will loop through each time step to calculate the hourly landslide susceptibility, which mainly includes the following tasks:
- Regrid ERA5 datasets
- Calculate subsurface flow depth
- Calculate factor of safety
- Calculate susceptibility
- Plot results


**Regrid ERA5 datasets**

The ERA5 precipitation and volumetric soil water datasets will be regridded in a higher resolution by using the Topography dataset as the template. The regrid_data( ) function will be used which is similar as the code shown in [Step2](#step2). As mentioned before, the volumetric soil water data is used for calculation and the precipitation data is mainly used to visualize the rainfall change during the time period. 

**Subsurface flow depth** 

The subsurface flow depth is calculated using the soil depth data and the ERA5 volumetric soil water datasets. There are four layers of the ERA5 volumetric soil water data (Layer 1: 0 - 7cm, Layer 2: 7 - 28cm, Layer 3: 28 - 100cm, Layer 4: 100 - 289cm). We first calculate the product of the soil depth at each layer and its corresponding volumetric soil water data, which represents the water depth at each soil layer. Then we add up the water depth values of all four layers, which will be an approximation of the subsurface flow depth. The cal_subsurface_flow_depth( ) function is used for calculation.

**Factor of safety**

In geological engineering, it is common to take the ratio of the resisting stresses to driving stresses. This ratio is called the factor of safety (FS). When FS is larger than 1, the slope should be stable, while if it is below 1, the driving stress exceeds the resistance and the slope is likely to fail. FS can be calculated with the following function, and cal_safety_factor( ) is implemented based on this function.

$$
FS = \frac{(C_r + C_s)/h_s\rho_sg}{sin\theta} + \frac{cos\theta tan\phi (1-\frac{h_w}{h_s}\rho_w / \rho_s)}{sin\theta}
$$

where, 
- Cr: root cohesion (Pa kg/ms^2)
- Cs: soil cohesion (Pa kg/ms^2)
- hs: soil depth (m)
- hw: subsurface flow depth (m)
- ρs: soil density (kg/m^3)
- ρw: water density (kg/m^3)
- g: gravity acceleration (m/s^2)
- θ: slope angle 
- φ: soil internal friction angle

**Susceptibility**

Susceptibility is the inverse of FS. When the susceptibility is larger than 1, it means that the slope of area is not stable and susceptible to landslide.  

$$
Susceptibility = \frac{1}{FS}
$$


**Plot Results**

At each time step, it makes four plots. The first plot is the landslide susceptibility at current time step. The second plot is the difference of the landslide susceptibility between the current and the last step. The third plot is the difference of the subsurface flow depth between the current and the last step. The fourth plot is the precipitation at the current time step. Please note that at the first time step, the second plot will not be applicable and the third plot will show the subsurface flow depth result.

In [ ]:
import cftime
from datetime import datetime
from matplotlib import colors

from utils import cal_subsurface_flow_depth, cal_safety_factor


# define ERA5 input coor
era5_y = np.flip(np.arange(era5_grid_shape[0])*era5_grid_spacing[0] + era5_grid_origin[0])
era5_x = np.arange(era5_grid_shape[1])*era5_grid_spacing[1] + era5_grid_origin[1]
era5_coor = {'lon': era5_x, 
             'lat': era5_y}

soil_water_layer = np.empty([4,*dem_grid_shape], dtype=era5.var_type('Volumetric soil water layer 1'))

# define mask
mask = (slope_angle==0)&(soil_depth>2.0)


# timing
print(datetime.now())

# calculation and plot
for time_step in range(0, 24): 
    time = cftime.num2date(era5.time, era5_time_unit, only_use_cftime_datetimes=False, only_use_python_datetimes=True)
    print(time_step,time)
     
    # regrid ERA5 precipitation 
    era5_prec = era5.get_value('Total precipitation')
    era5_prec_2D = era5_prec.reshape(era5_grid_shape)
    era5_prec_data = regrid_data(era5_prec_2D, era5_coor, dest_coor)
    
    # regrid ERA5 volumetric soil water
    for i in range(0,4):
        var_name = 'Volumetric soil water layer {}'.format(i+1)

        # get original data
        era5_soil_water = era5.get_value(var_name)
        era5_soil_water_2D = era5_soil_water.reshape(era5_grid_shape)

        # regrid
        soil_water_layer[i] = regrid_data(era5_soil_water_2D, era5_coor, dest_coor)
       
    # calculate subsurface flow depth
    subsurface_flow_depth = cal_subsurface_flow_depth(soil_depth, soil_water_layer)
    
    # calculate FS
    safety_factor = cal_safety_factor(slope_angle, subsurface_flow_depth, soil_depth, 
                                      root_cohesion=5000, soil_cohesion=5000, soil_bulk_density=1300,
                                      soil_internal_friction_angle=35)
    
    # calculate susceptibility
    susceptibility = 1.0 / safety_factor
    susceptibility = np.where(~mask, susceptibility, np.nan)
       
    
    # plot susceptibility
    fig = plt.figure(figsize=(16,15))
    plt.rcParams["figure.autolayout"] = True
    
    nrows, ncols = 4,1
    ax_sus = fig.add_subplot(nrows, ncols, 1)
    ax_diff = fig.add_subplot(nrows, ncols, 2)
    ax_prcp = fig.add_subplot(nrows, ncols, 4)
    ax_soil = fig.add_subplot(nrows, ncols, 3)
    

    # susceptibility plot 
    divnorm=colors.TwoSlopeNorm(vcenter=1.0)
    im_sus=ax_sus.imshow(susceptibility, cmap='BrBG_r', norm=divnorm, extent=dem_extent)
    fig.colorbar(im_sus, ax=ax_sus)
    ax_sus.title.set_text('Susceptibility of landslide at {}'.format(time))
    ax_sus.set_xlabel('longitude [degree_east]')
    ax_sus.set_ylabel('latitude [degree_north]')
    
    # difference of susceptibility plot
    if time_step >0:
        diff2 = susceptibility - reference2      
        
        divnorm=colors.TwoSlopeNorm(vcenter=0.0)       
        im_diff=ax_diff.imshow(diff2, cmap='BrBG_r', norm=divnorm, extent=dem_extent)
        fig.colorbar(im_diff, ax=ax_diff)
        ax_diff.title.set_text('Difference of susceptibility')
    else:
        ax_diff.axis('off')
    
    ax_diff.set_xlabel('longitude [degree_east]')
    ax_diff.set_ylabel('latitude [degree_north]')
    reference2 = np.copy(susceptibility)  
    
    # precipitation plot
    vmax = None if era5_prec_data.max() != 0 else 0.0001   
    im_prcp = ax_prcp.imshow(era5_prec_data, cmap='Blues', vmin=0.0, vmax=vmax, extent=dem_extent)
    fig.colorbar(im_prcp, ax=ax_prcp)
    ax_prcp.title.set_text('Precipitation (m)')
    ax_prcp.set_xlabel('longitude [degree_east]')
    ax_prcp.set_ylabel('latitude [degree_north]')
    
    # subsurface flow depth plot
    if time_step==0:
        im_data = subsurface_flow_depth
        ax_soil.title.set_text('Subsurface flow depth')
    else:
        im_data = subsurface_flow_depth - reference3
        ax_soil.title.set_text('Difference of subsurface flow depth')
    
    ax_soil.set_xlabel('longitude [degree_east]')
    ax_soil.set_ylabel('latitude [degree_north]')
    
    divnorm=colors.TwoSlopeNorm(vcenter=0.0) 
    im_soil = ax_soil.imshow(im_data, cmap='RdBu', norm=divnorm,  extent=dem_extent)

    fig.colorbar(im_soil, ax=ax_soil)
    
    reference3 = np.copy(subsurface_flow_depth)   
        
    # save plot    
    plt.close(fig)
    fig.savefig(os.path.join(results_dir, 'sus_{}.png'.format(time_step)))
    
    # update time step
    era5.update()
    

print(datetime.now())

print('calculation is done')

<a id='step4'></a>
## Step 4 Visualize Results

Run the cells below and it will show a short video. At each time step, the area with brown color in the first plot has susceptibility value lager than 1, which indicates that the slope is unstable and landslide event may occur. In the second and third plots, you'll find that when the subsurface flow depth increases (blue color), the land susceptibility increase (brown color). When the subsurface flow depth decreases (red color), the land susceptibility decreases (green color). 

In [ ]:
import imageio
import os

# Make a short video
img_files = [os.path.join(results_dir, file) for file in os.listdir(results_dir) if '.png' in file]
img_files.sort(key=lambda x: os.path.getmtime(x))

with imageio.get_writer(os.path.join(results_dir,'landslide.mp4'), mode='I', fps=1) as writer:
    for f in img_files:
        im = imageio.imread(os.path.join(results_dir, f))
        writer.append_data(im)

writer.close()

 <video controls src="./results_puerto_rico/landslide.mp4" />

## References
- Strauch, R., Istanbulluoglu, E., Nudurupati, S. S., Bandaragoda, C., Gasparini, N. M., and Tucker, G. E. (2018), A hydroclimatological approach to predicting regional landslide probability using Landlab, Earth Surf. Dynam., 6, 49–75, https://doi.org/10.5194/esurf-6-49-2018
- Montgomery, D. R., and Dietrich, W. E. (1994), A physically based model for the topographic control on shallow landsliding, Water Resour. Res., 30( 4), 1153– 1171, doi:10.1029/93WR02979.